In [1]:
import os, sys
import argparse
import json

p_prompt = f"""
Passage: <P> Given the provided passage, generate 3 similar passages on related topic: <T>
"""

q_prompt = """
<P> Review the given passages and answer a specific and detailed query. {'Query: Your query here.'}”
"""

parser = argparse.ArgumentParser(description = "Prompting with ExLlamaV2")
parser.add_argument("--dataset", type = str, default ='nq')
parser.add_argument("--seed", type = int, default = 2023)
parser.add_argument("--topk", type=int, default=10)
parser.add_argument("--model_dir", type=str, default="/root/Mistral-7B-instruct-exl2")
parser.add_argument("--p_prompt", type=str, default=p_prompt)
parser.add_argument("--q_prompt", type=str, default=q_prompt)

args = parser.parse_args([])

In [2]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{args.dataset}.zip"
data_path = util.download_and_unzip(url, './data/')

corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

/root/miniconda3/envs/rerank/lib/python3.10/site-packages/beir/util.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
./data/nq.zip: 100%|██████████| 475M/475M [00:49<00:00, 10.0MiB/s]  
100%|██████████| 2681468/2681468 [00:12<00:00, 213131.69it/s]


In [ ]:
# # Download & Setup ElasticSearch

# %%bash

# wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
# wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
# tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
# sudo chown -R daemon:daemon elasticsearch-7.9.2/
# shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

# %%bash --bg

# sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

# %%bash

# ps -ef | grep elasticsearch

# %%bash

# curl -sX GET "localhost:9200/"

In [5]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

hostname = "localhost"
index_name = args.dataset
initialize = True

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
retriever = EvaluateRetrieval(model)

# results = retriever.retrieve(corpus, queries)
# # ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
# output = {'retrieval': EvaluateRetrieval.evaluate(qrels, results, retriever.k_values)}

In [6]:
def save_json(results, file_name:str):
    json_path = f'./output/{args.dataset}_{file_name}.json'
    with open(json_path, 'w') as f:
        json.dump(results, f, indent=4)

def read_json(json_path:str):
    with open(json_path, 'r') as f:
        data = f.read()
        output = json.loads(data)
    return output

In [14]:
save_json(results, 'bm25_es_retrieved')
save_json(output, 'eval_output')

In [7]:
results = read_json(f'./output/{args.dataset}_bm25_es_retrieved.json')
output = read_json(f'./output/{args.dataset}_eval_output.json')

In [8]:
from exllamav2 import(
    ExLlamaV2,
    ExLlamaV2Config,
    ExLlamaV2Cache,
    ExLlamaV2Tokenizer,
)

from exllamav2.generator import (
    ExLlamaV2BaseGenerator,
    ExLlamaV2Sampler
)

# from beir.reranking import Rerank
from typing import List, Tuple
import torch
import torch.nn.functional as F

In [9]:
# https://github.com/beir-cellar/beir/blob/main/beir/reranking/rerank.py
from typing import Dict, List

class Rerank:
    def __init__(self, model, batch_size: int = 128, **kwargs):
        self.model = model
        self.batch_size = batch_size
        self.rerank_results = {}
    
    def rerank(self,
               corpus: Dict[str, Dict[str, str]],
               queries: Dict[str, str],
               results: Dict[str, Dict[str, float]],
               top_k: int) -> Dict[str, Dict[str, float]]:
        
        sentence_pairs, pair_ids = [], []
        
        for query_id in results:
            if len(results[query_id]) > top_k:
                for (doc_id, _) in sorted(results[query_id].items(), key=lambda item: item[1], reverse=True)[:top_k]:
                    pair_ids.append([query_id, doc_id])
                    corpus_text = {'title': corpus[doc_id].get("title", "").strip(), 'text': corpus[doc_id].get("text", "").strip()}
                    sentence_pairs.append([queries[query_id], corpus_text])
                else:
                    for doc_id in results[query_id]:
                        pair_ids.append([query_id, doc_id])
                        corpus_text = {'title': corpus[doc_id].get("title", "").strip(), 'text': corpus[doc_id].get("text", "").strip()}
                        # corpus_text = (corpus[doc_id].get("title", "") + " " + corpus[doc_id].get("text", "")).strip()
                        sentence_pairs.append([queries[query_id], corpus_text])
        
        #### Starting to Rerank using cross-attention
        rerank_scores = [float(score) for score in self.model.predict(sentence_pairs, batch_size=self.batch_size)]
        
        #### Reranking results
        self.rerank_results = {query_id: {} for query_id in results}
        for pair, score in zip(pair_ids, rerank_scores):
            query_id, doc_id = pair[0], pair[1]
            self.rerank_results[query_id][doc_id] = score
        
        return self.rerank_results

In [ ]:
# https://github.com/beir-cellar/beir/wiki/Evaluate-your-custom-model
# https://github.com/Muennighoff/sgpt/blob/main/crossencoder/beir/sgptce.py
# https://github.com/beir-cellar/beir/blob/main/examples/retrieval/evaluation/reranking/evaluate_bm25_ce_reranking.py
# https://github.com/beir-cellar/beir/blob/main/beir/reranking/models/mono_t5.py

class ExLlamaV2Reranker:
    def __init__(self, p_prompt=args.p_prompt, q_prompt=args.q_prompt, model_dir=args.model_dir, seed=args.seed, max_new_tokens=400, **kwargs):
        self.p_prompt = p_prompt
        self.q_prompt = q_prompt
        self.seed = seed
        self.max_new_tokens = max_new_tokens
        
        self.config = ExLlamaV2Config()
        self.config.model_dir = model_dir
        self.config.prepare()
        
        self.model = ExLlamaV2(self.config)
        if not self.model.loaded:
            self.cache = ExLlamaV2Cache(self.model, lazy = True)
            self.model.load_autosplit(self.cache)
            
        self.tokenizer = ExLlamaV2Tokenizer(self.config)
        self.generator = ExLlamaV2BaseGenerator(self.model, self.cache, self.tokenizer)
        self.generator.warmup()
        
        self.settings = ExLlamaV2Sampler.Settings()
        self.settings.temperature = 0.6
        self.settings.top_k = 50
        self.settings.top_p = 0.9
        self.settings.token_repetition_penalty = 1.15
        self.settings.disallow_tokens(self.tokenizer, [self.tokenizer.eos_token_id])
    
    def predict(self, sentences: List[Tuple[str,str]], batch_size: int, **kwags) -> List[float]:
        scores = []
        for query, psg in sentences:
            self.p_prompt = self.p_prompt.replace('<P>', psg['text']).replace('<T>', psg['title']).strip()
            generated = self.generator.generate_simple(self.p_prompt, self.settings, self.max_new_tokens, seed=self.seed)
            self.q_prompt = self.q_prompt.replace('<P>', generated).strip()
            with torch.inference_mode():
                input_ids = self.tokenizer.encode(self.q_prompt)
                # input_ids = input_ids.shape[-1]
                # self.cache.current_seq_len = 0
                logits = self.model.forward(input_ids[:, -1:], self.cache)
                logits = logits[:, :-1, :]
                logits = logits.float() + 1e-10
                log_probs = F.log_softmax(logits, dim=-1)
            scores.append(log_probs)
        
        assert len(scores) == len(sentences)
        return scores

In [43]:
reranker = Rerank(ExLlamaV2Reranker(), batch_size=128)
rerank_results = reranker.rerank(corpus, queries, results, top_k=100)

KeyboardInterrupt: 

In [ ]:
# save output
output['reranking'] = EvaluateRetrieval.evaluate(qrels, rerank_results, retriever.k_values)

with open(f'/content/{args.dataset}_eval_ouput.json', 'w') as f:
    json.dump(output, f, indent=4)

In [ ]:
import random
import logging

top_k = 10


query_id, ranking_scores = random.choice(list(rerank_results.items()))
scores_sorted = sorted(ranking_scores.items(), key=lambda item: item[1], reverse=True)
logging.info("Query : %s\n" % queries[query_id])

for rank in range(top_k):
  doc_id = scores_sorted[rank][0]
  # Format: Rank x: ID [Title] Body
  logging.info("Rank %d: %s [%s] - %s\n" % (rank+1, doc_id, corpus[doc_id].get("title"), corpus[doc_id].get("text")))